# Getting Just the Matching Model Paths
* Creates a list of dictionaries with the matching models for each experiment.
* Data is then used in zec_02_regridding...

* 6th May
     * Updateing to only use dictionary. Previous version had a csv file with only tas. Making a new version that stores all output.

TODO:
* Currently this is only really working for piControl and 1000pgc. Need to add the 1%CO2 runs to this
* Will eventually need to create the flows for other variables. This is perhaps partially setup.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from importlib import reload
from glob import glob
import json
from pprint import pprint
sys.path.append(os.path.join(os.getcwd(), 'Documents/PhD'))
sys.path.append('modules')
import constants

from typing import List

In [3]:
constants.IMAGE_SAVE_DIR_INIT

'/g/data/w40/ab2313/images/PhD/init'

# Paths

The code below can be used to ge the paths of all the files. However, this can be ery slow (3 hours) for piControl. 

In [20]:
# TODO: This should be a dict not a csv so you don't have to drop nans
# fpaths_df = pd.read_csv(os.path.join(os.getcwd(),  'Documents/PhD', 'data/fpaths_picontrol.csv'))#[['picontrol', '1000pgc']]
# fpaths_df.head()
# fpaths_dict = fpaths_df.to_dict(orient='list')
# fpaths_dict['1000pgc'] = [v for v in fpaths_dict['1000pgc'] if isinstance(v, str)]
# fpaths_dict['1pct_rad'] = [v for v in fpaths_dict['1pct_rad'] if isinstance(v, str)]

# fpaths_dict.keys()

# unique_picontrol_paths = np.unique(list(map(os.path.dirname, fpaths_dict['picontrol'])))
# unique_picontrol_paths[:5]

In [37]:
with open(os.path.join(os.getcwd(),  'Documents/PhD/data', 'raw_paths.json'), 'r') as file:
    fpaths_dict = json.loads(file.read())

In [38]:
variable = 'tas'

In [39]:
fpaths_dict.keys()

dict_keys(['picontrol', '1000pgc', '1pct_rad', '1pct_bgc'])

In [40]:
print(fpaths_dict['picontrol'][variable][:3], fpaths_dict['1000pgc'][variable][:3],
      fpaths_dict['1pct_rad'][variable][:3], fpaths_dict['1pct_bgc'][variable][:3],
      sep='\n\n\n')

['/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_314801-319612.nc', '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_246901-249512.nc', '/g/data/oi10/replicas/CMIP6/CMIP/INM/INM-CM5-0/piControl/r1i1p1f1/Amon/tas/gr1/v20190619/tas_Amon_INM-CM5-0_piControl_r1i1p1f1_gr1_236901-246812.nc']


['/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815/tas_Amon_MPI-ESM1-2-LR_esm-1pct-brch-1000PgC_r1i1p1f1_gn_207501-209412.nc', '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815/tas_Amon_MPI-ESM1-2-LR_esm-1pct-brch-1000PgC_r1i1p1f1_gn_205501-207412.nc', '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815/tas_Amon_MPI-ESM1-2-LR_esm-1pct-brch-1000PgC_r1i1p1f1_gn_199501-201

In [59]:
def zecmip_open_matching_path(fpath: str, 
                            path_list: List[str],
                            experiment:str='esm-1pct-brch-1000PgC', debug=False) -> str:
    
    '''This function takes a path, and finds the matching path for piControl from a list of paths
    containing piControl paths.
    
    Parameters
    ----------
    fpath: the path to the dataset that you want to match to piControl.
    unique_picontrol_paths: paths to different piControl runs.
    experiment: the experiment of what you are matching.
    
    
    
    Sample of split path
    fpath = '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429'
    fpath.split('/')
    0-, 1-g, 2-data, 3-oi10, 4-replicas, 5-CMIP6, 6-C4MIP, 7-CCCma, 8-CanESM5, 9-esm-1pct-brch-1000PgC, 10-r1i1p2f1, 11-Amon,       12-tas, 13-gn, 14-v20190429
    
    '''
    import re
    print(f'Attempting to open matching path for:\n{fpath}')
        
    path_branch = '/'.join(fpath.split('/')[7:-2])
    if debug: print(f'{path_branch=}')
    path_branch = path_branch.replace(experiment, '\w+')#'piControl|esm-piControl')
    if debug: print(f'Searching for branch containing\n{path_branch}')
        
    PATH_FOUND = False
    for path in path_list:
        #if path_branch in pi_path:
        core_path = '/'.join(path.split('/')[7:-2])
        if debug:print(f'{path_branch=} - {core_path=}')
        if re.search(path_branch, core_path):
            PATH_FOUND = True
            if debug: print(f'Found branch:\n{path_branch}')
            break
    if not PATH_FOUND: return np.nan
    if debug:  print(f'Found path:\n{picontrol_path}')
    return path

In [75]:
def zecmip_open_matching_path(fpath:str, path_list: List[str]) -> str:
    split_path = fpath.split('/')
    model = split_path[8]; ensemble_member = split_path[10]
    print(f'Attemping to find {model=} and {ensemble_member=}')
    for potential_match in path_list:
        if model in potential_match and ensemble_member in potential_match: return potential_match
    print('No match found')
    return np.nan

In [76]:
# Unique abspath (no filename).
base_fpaths_1000PgC = np.sort(np.unique(list(map(os.path.dirname,fpaths_dict['1000pgc'][variable]))))
base_fpaths_1000PgC

array(['/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r3i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r4i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r5i1p2f1/Amon/tas/gn/v20190429',
       '/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200622',
       '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200210',
       '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r2i1p1f2/Amon/tas/gn/v20200106',
       '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r3i1p1f2/Amon/tas/gn/v

In [93]:
len(path_matches)

5

In [94]:
pprint(path_matches)

[{'/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200622': '/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/1pctCO2-bgc/r1i1p1f2/Amon/tas/gn/v20191129/tas_Amon_MIROC-ES2L_1pctCO2-bgc_r1i1p1f2_gn_185001-199912.nc'},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200210': '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/1pctCO2-bgc/r1i1p1f2/Amon/tas/gn/v20190723/tas_Amon_UKESM1-0-LL_1pctCO2-bgc_r1i1p1f2_gn_195001-199912.nc'},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815': '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/1pctCO2-bgc/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-LR_1pctCO2-bgc_r1i1p1f1_gn_193001-194912.nc'},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/NCAR/CESM2/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191119': '/g/data/oi10/replicas/CMIP6/C4MIP/NCAR/CESM2/1pctCO2-bgc/r1i1p1f1/Amon/tas/gn/v201

In [92]:
path_matches = []
for fpath in base_fpaths_1000PgC:
    match = zecmip_open_matching_path(fpath, fpaths_dict['1pct_bgc'][variable])
    if isinstance(match, str): path_matches.append({fpath:match})

Attemping to find model='CanESM5' and ensemble_member='r1i1p2f1'
No match found
Attemping to find model='CanESM5' and ensemble_member='r2i1p2f1'
No match found
Attemping to find model='CanESM5' and ensemble_member='r3i1p2f1'
No match found
Attemping to find model='CanESM5' and ensemble_member='r4i1p2f1'
No match found
Attemping to find model='CanESM5' and ensemble_member='r5i1p2f1'
No match found
Attemping to find model='MIROC-ES2L' and ensemble_member='r1i1p1f2'
Attemping to find model='UKESM1-0-LL' and ensemble_member='r1i1p1f2'
Attemping to find model='UKESM1-0-LL' and ensemble_member='r2i1p1f2'
No match found
Attemping to find model='UKESM1-0-LL' and ensemble_member='r3i1p1f2'
No match found
Attemping to find model='UKESM1-0-LL' and ensemble_member='r4i1p1f2'
No match found
Attemping to find model='MPI-ESM1-2-LR' and ensemble_member='r1i1p1f1'
Attemping to find model='GISS-E2-1-G-CC' and ensemble_member='r1i1p1f1'
No match found
Attemping to find model='CESM2' and ensemble_member='

In [62]:
path_matches

[{'/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/Amon/tas/gn/v20190429': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r3i1p2f1/Amon/tas/gn/v20190429': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r4i1p2f1/Amon/tas/gn/v20190429': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r5i1p2f1/Amon/tas/gn/v20190429': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200622': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200210': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r2i1p1f2/Amon/tas/gn/v20200106': nan},
 {'/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r3i1p1f2/Amon/tas/g

In [61]:
path_matches = []
for fpath in base_fpaths_1000PgC:
    match = zecmip_open_matching_path(fpath, fpaths_dict['picontrol'][variable], 'piControl')
    path_matches.append({fpath:match})

Attempting to open matching path for:
/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429
Attempting to open matching path for:
/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r2i1p2f1/Amon/tas/gn/v20190429
Attempting to open matching path for:
/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r3i1p2f1/Amon/tas/gn/v20190429
Attempting to open matching path for:
/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r4i1p2f1/Amon/tas/gn/v20190429
Attempting to open matching path for:
/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r5i1p2f1/Amon/tas/gn/v20190429
Attempting to open matching path for:
/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200622
Attempting to open matching path for:
/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200210
Attempting to open ma

In [42]:
# These are all of the base models - models may have different experminents (r1i1p2f1, r2i1p2f1)
base_group = np.unique(['/'.join(f.split('/')[:10])  for f in base_fpaths_1000PgC])
base_group

array(['/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC',
       '/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/esm-1pct-brch-1000PgC',
       '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC',
       '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC',
       '/g/data/oi10/replicas/CMIP6/C4MIP/NASA-GISS/GISS-E2-1-G-CC/esm-1pct-brch-1000PgC',
       '/g/data/oi10/replicas/CMIP6/C4MIP/NCAR/CESM2/esm-1pct-brch-1000PgC',
       '/g/data/oi10/replicas/CMIP6/C4MIP/NCC/NorESM2-LM/esm-1pct-brch-1000PgC',
       '/g/data/oi10/replicas/CMIP6/C4MIP/NOAA-GFDL/GFDL-ESM4/esm-1pct-brch-1000PgC'],
      dtype='<U80')

In [43]:
# Get one experiment for each modelling group
needed_experiments = []
# Looping through all the base paths
for base_group_single in base_group:
    
    # All the experiments that have this base path
    experiments = [f for f in base_fpaths_1000PgC if base_group_single in f]
    # Get just the first one available
    needed_experiments.append(experiments[0])
    
needed_experiments

['/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
 '/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200622',
 '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200210',
 '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815',
 '/g/data/oi10/replicas/CMIP6/C4MIP/NASA-GISS/GISS-E2-1-G-CC/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191202',
 '/g/data/oi10/replicas/CMIP6/C4MIP/NCAR/CESM2/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191119',
 '/g/data/oi10/replicas/CMIP6/C4MIP/NCC/NorESM2-LM/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191108',
 '/g/data/oi10/replicas/CMIP6/C4MIP/NOAA-GFDL/GFDL-ESM4/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gr1/v20180701']

# Old

In [12]:
needed_model_dict = {v.split('/')[8]:v for v in needed_experiments}
needed_model_dict

{'CanESM5': '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
 'MIROC-ES2L': '/g/data/oi10/replicas/CMIP6/C4MIP/MIROC/MIROC-ES2L/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200622',
 'UKESM1-0-LL': '/g/data/oi10/replicas/CMIP6/C4MIP/MOHC/UKESM1-0-LL/esm-1pct-brch-1000PgC/r1i1p1f2/Amon/tas/gn/v20200210',
 'MPI-ESM1-2-LR': '/g/data/oi10/replicas/CMIP6/C4MIP/MPI-M/MPI-ESM1-2-LR/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20190815',
 'GISS-E2-1-G-CC': '/g/data/oi10/replicas/CMIP6/C4MIP/NASA-GISS/GISS-E2-1-G-CC/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191202',
 'CESM2': '/g/data/oi10/replicas/CMIP6/C4MIP/NCAR/CESM2/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191119',
 'NorESM2-LM': '/g/data/oi10/replicas/CMIP6/C4MIP/NCC/NorESM2-LM/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gn/v20191108',
 'GFDL-ESM4': '/g/data/oi10/replicas/CMIP6/C4MIP/NOAA-GFDL/GFDL-ESM4/esm-1pct-brch-1000PgC/r1i1p1f1/Amon/tas/gr1/v20180701'}

In [13]:
zecmip_path_dict = {'1000pgc': {'all': unique_1000PgC_paths.tolist(), 'needed': needed_model_dict}
                    , 'picontrol': unique_picontrol_paths.tolist()}
zecmip_path_dict.keys()

dict_keys(['1000pgc', 'picontrol'])

In [15]:
pprint(zecmip_path_dict)

{'1000pgc': {'all': ['/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/co2/gn/v20190429',
                     '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/pr/gn/v20190429',
                     '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Amon/tas/gn/v20190429',
                     '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Lmon/mrso/gn/v20190429',
                     '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/Omon/tos/gn/v20190429',
                     '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/day/pr/gn/v20190429',
                     '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/day/tas/gn/v20190429',
                     '/g/data/oi10/replicas/CMIP6/C4MIP/CCCma/CanESM5/esm-1pct-brch-1000PgC/r1i1p2f1/day/tasmax/gn/v20190429',
 

In [10]:
with open('data/zecmip_model_paths.json', 'w') as f:
    json.dump(zecmip_path_dict, f)